## Lab| Customer Analysis Case Study| Modeling

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import boxcox
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings('ignore')

In [9]:
cutsomer_df=pd.read_csv("Data/clean_data.csv" )

### Xy Split

In [11]:
X=customers_df.drop('total_claim_amount', axis=1)
y=customers_df.total_claim_amount

In [12]:
X

,state,customer_lifetime_value,response,coverage,employmentstatus,gender,income,location_code,marital_status,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Arizona,4809.0,No,Basic,Employed,M,48029,Suburban,Married,61,7.0,52,0.0,3,Corporate Auto,Corporate,Offer3,Agent,Four-Door Car,Medsize
2,Washington,14948.0,No,Basic,Employed,M,22139,Suburban,Single,100,34.0,31,0.0,2,Personal Auto,Personal,Offer3,Call Center,Four-Door Car,Medsize
3,Oregon,22332.0,Yes,Extended,Employed,M,49078,Suburban,Single,97,10.0,3,0.0,2,Corporate Auto,Corporate,Offer2,Branch,Four-Door Car,Medsize
6,California,5035.0,No,Basic,Employed,F,37405,Urban,Married,63,8.0,99,3.0,3,Corporate Auto,Corporate,Offer2,Branch,Four-Door Car,Medsize
7,California,4956.0,No,Basic,Employed,M,87197,Urban,Single,63,35.0,45,0.0,3,Personal Auto,Personal,Offer2,Branch,Two-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10902,California,3579.0,No,Extended,Employed,F,28304,Suburban,Married,91,10.0,30,2.0,1,Personal Auto,Personal,Offer2,Agent,Four-Door Car,Medsize
10903,Arizona,2772.0,No,Basic,Employed,M,59855,Suburban,Married,74,30.0,82,4.0,1,Personal Auto,Personal,Offer2,Branch,Two-Door Car,Medsize
10906,Oregon,5259.0,No,Basic,Employed,F,61146,Urban,Married,65,7.0,68,0.0,3,Personal Auto,Personal,Offer2,Branch,Four-Door Car,Medsize
10907,Arizona,23893.0,No,Extended,Employed,F,39837,Rural,Married,201,11.0,63,0.0,2,Corporate Auto,Corporate,Offer1,Web,Luxury Car,Medsize


In [13]:
numericals_features =X._get_numeric_data()
# numericals =data.select_dtypes(np.number)
numericals_features.head()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,4809.0,48029,61,7.0,52,0.0,3
2,14948.0,22139,100,34.0,31,0.0,2
3,22332.0,49078,97,10.0,3,0.0,2
6,5035.0,37405,63,8.0,99,3.0,3
7,4956.0,87197,63,35.0,45,0.0,3


In [14]:
numericals_features.describe()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
count,6675.000000,6675.000000,6675.000000,6675.000000,6675.000000,6675.000000,6675.000000
mean,8136.941573,50593.521049,92.924345,15.050037,47.835206,0.383820,2.032509
std,6946.779205,24278.962458,34.521787,10.073042,27.912248,0.905702,0.865372
min,2120.000000,10037.000000,61.000000,0.000000,0.000000,0.000000,1.000000
25%,4098.500000,28573.000000,68.000000,6.000000,24.000000,0.000000,1.000000
50%,5825.000000,48207.000000,83.000000,14.000000,47.000000,0.000000,2.000000
75%,9076.000000,70263.000000,109.000000,23.000000,71.000000,0.000000,3.000000
max,83325.000000,99981.000000,297.000000,35.000000,99.000000,5.000000,3.000000


In [17]:
categoricals_features = X.select_dtypes(object)
categoricals_features

,state,response,coverage,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Arizona,No,Basic,Employed,M,Suburban,Married,Corporate Auto,Corporate,Offer3,Agent,Four-Door Car,Medsize
2,Washington,No,Basic,Employed,M,Suburban,Single,Personal Auto,Personal,Offer3,Call Center,Four-Door Car,Medsize
3,Oregon,Yes,Extended,Employed,M,Suburban,Single,Corporate Auto,Corporate,Offer2,Branch,Four-Door Car,Medsize
6,California,No,Basic,Employed,F,Urban,Married,Corporate Auto,Corporate,Offer2,Branch,Four-Door Car,Medsize
7,California,No,Basic,Employed,M,Urban,Single,Personal Auto,Personal,Offer2,Branch,Two-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10902,California,No,Extended,Employed,F,Suburban,Married,Personal Auto,Personal,Offer2,Agent,Four-Door Car,Medsize
10903,Arizona,No,Basic,Employed,M,Suburban,Married,Personal Auto,Personal,Offer2,Branch,Two-Door Car,Medsize
10906,Oregon,No,Basic,Employed,F,Urban,Married,Personal Auto,Personal,Offer2,Branch,Four-Door Car,Medsize
10907,Arizona,No,Extended,Employed,F,Rural,Married,Corporate Auto,Corporate,Offer1,Web,Luxury Car,Medsize


### Categorical Variables Transformation and reunion all_features

In [18]:
# get dummies pandas
categoricals_features=pd.get_dummies(categoricals_features, drop_first=True)
categoricals_features

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,coverage_Extended,coverage_Premium,employmentstatus_Employed,employmentstatus_Medical Leave,employmentstatus_Retired,gender_M,location_code_Suburban,location_code_Urban,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Personal,policy_Special,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0
3,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
6,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
7,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10902,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0
10903,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0
10906,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0
10907,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0


In [20]:
all_features=pd.concat([numericals_features, categoricals_features], axis=1) # concat numerical and categorical transformations
all_features

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,coverage_Extended,coverage_Premium,employmentstatus_Employed,employmentstatus_Medical Leave,employmentstatus_Retired,gender_M,location_code_Suburban,location_code_Urban,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Personal,policy_Special,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
0,4809.0,48029,61,7.0,52,0.0,3,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,14948.0,22139,100,34.0,31,0.0,2,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0
3,22332.0,49078,97,10.0,3,0.0,2,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
6,5035.0,37405,63,8.0,99,3.0,3,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
7,4956.0,87197,63,35.0,45,0.0,3,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10902,3579.0,28304,91,10.0,30,2.0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0
10903,2772.0,59855,74,30.0,82,4.0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0
10906,5259.0,61146,65,7.0,68,0.0,3,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0
10907,23893.0,39837,201,11.0,63,0.0,2,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0


### Train Test Split

In [21]:
X_train, X_test, y_train, y_test=train_test_split(all_features, y, test_size=.20, random_state=42)

In [22]:
X_train

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,coverage_Extended,coverage_Premium,employmentstatus_Employed,employmentstatus_Medical Leave,employmentstatus_Retired,gender_M,location_code_Suburban,location_code_Urban,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Personal,policy_Special,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
7420,4864.0,88079,61,16.0,37,0.0,3,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,1,0
3822,9594.0,27443,86,3.0,1,2.0,2,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1338,7233.0,46621,61,20.0,31,0.0,2,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
585,4782.0,81687,61,7.0,77,0.0,3,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
2947,18592.0,14209,62,34.0,36,0.0,2,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6247,6898.0,70950,171,6.0,12,0.0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0
8548,4529.0,94757,112,0.0,25,0.0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
8599,10214.0,60078,127,23.0,60,0.0,3,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
8854,3490.0,90985,87,33.0,39,0.0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0


#### Standardize

In [23]:
std_scaler=StandardScaler().fit(X_train) 

X_train_scaled=std_scaler.transform(X_train)

In [24]:
X_test_scaled=std_scaler.transform(X_test)

### Modeling

In [67]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [68]:
model_list = [LinearRegression(), Lasso(), Ridge(), KNeighborsRegressor()]

In [92]:
def model_application(model_list):
    r2 = dict()
    for model in model_list:
        model.fit(X_train_scaled, y_train)
        y_pred=model.predict(X_test_scaled)
        r2[model] = r2_score(y_test, y_pred)
        print(f"{model}:\n Train: {model.score(X_train_scaled, y_train).round(4)},\n Test: {model.score(X_test_scaled, y_test).round(4)},\n Rsquared: {r2[model].round(4)} \n")

In [93]:
model_application(model_list)

LinearRegression():
 Train: 0.7989,
 Test: 0.8169,
 Rsquared: 0.8169 

Lasso():
 Train: 0.7984,
 Test: 0.8174,
 Rsquared: 0.8174 

Ridge():
 Train: 0.7989,
 Test: 0.8169,
 Rsquared: 0.8169 

KNeighborsRegressor():
 Train: 0.7176,
 Test: 0.5571,
 Rsquared: 0.5571 

